In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmcv.runner import load_checkpoint
from tqdm.notebook import tqdm
import torch
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.6.0].
device:[cuda:1].


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/swin/cascade_mask_rcnn_swin_base-Copy1.py')

PREFIX = '../../input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_all.json'
cfg.data.train.seg_prefix=PREFIX  

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'

cfg.data.samples_per_gpu = 4

cfg.seed=42
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/swin+cascade'
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [3]:
import os
from mmcv.runner import load_checkpoint
model = build_detector(cfg.model)

2021-05-20 02:43:11,981 - mmdet - INFO - load model from: /home/ubuntu/CUAI_2021/Advanced_Won_Cho/code/Swin-Transformer-Object-Detection/work_dirs/swin+cascade/swin_small_patch4_window7_224.pth
2021-05-20 02:43:14,569 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=5.33s)
creating index...
index created!


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/albumentations/augmentations/transforms.py:2611: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [5]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-20 02:43:25,863 - mmdet - INFO - load checkpoint from /home/ubuntu/CUAI_2021/Advanced_Won_Cho/code/Swin-Transformer-Object-Detection/work_dirs/swin+cascade/cascade_mask_rcnn_swin_small_patch4_window7.pth
2021-05-20 02:43:25,865 - mmdet - INFO - Use load_from_local loader


Done (t=1.41s)
creating index...
index created!


2021-05-20 02:43:26,241 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.0.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([4, 1024]).
size mismatch for roi_head.bbox_head.0.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([4]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: cop

2021-05-20 02:43:26,255 - mmdet - INFO - Start running, host: ubuntu@nipa2021-1083, work_dir: /home/ubuntu/CUAI_2021/Advanced_Won_Cho/code/Swin-Transformer-Object-Detection/work_dirs/swin+cascade
2021-05-20 02:43:26,258 - mmdet - INFO - workflow: [('train', 1)], max: 60 epochs
/home/ubuntu/CUAI_2021/Advanced_Won_Cho/code/Swin-Transformer-Object-Detection/mmdet/core/evaluation/eval_hooks.py:111: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/instaboostfast/pycococreater.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  contours = np.subtract(contours, 1)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/instaboos

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 4.0 task/s, elapsed: 162s, ETA:     0s

2021-05-20 18:16:32,042 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=1.22s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.37s).
Accumulating evaluation results...
DONE (t=1.31s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.526
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.697
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.559
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.231
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.528
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.785
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.619
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.619
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.619
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]

2021-05-20 18:16:45,784 - mmdet - INFO - Now best checkpoint is epoch_36.pth.Best bbox_mAP_50 is 0.6970
2021-05-20 18:16:45,812 - mmdet - INFO - Epoch(val) [36][818]	bbox_mAP: 0.5260, bbox_mAP_50: 0.6970, bbox_mAP_75: 0.5590, bbox_mAP_s: 0.2310, bbox_mAP_m: 0.5280, bbox_mAP_l: 0.7850, bbox_mAP_copypaste: 0.526 0.697 0.559 0.231 0.528 0.785
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/instaboostfast/pycococreater.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  contours = np.subtract(contours, 1)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/instaboostfast/pycococreater.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different

KeyboardInterrupt: 